In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import set_seed

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
templates = ['满意','可以','好','行','喜欢','高兴','开心']
task = 'douban'
# mode = 'dataset'
mode = 'instance'
random_flag = False
seed = 0

set_seed = (seed)

In [65]:
accs = {}
for template in templates:
    df = pd.read_csv(f'./zeroshot_results/{task}_{template}.csv',index_col=0)
    accs[template] = df

In [66]:
ppls = {}
for template in templates:
    df_neg = pd.read_csv(f'./datasets_ppl_score/{task}_ppl_temp_不{template}。.csv',index_col=0)
    df_pos = pd.read_csv(f'./datasets_ppl_score/{task}_ppl_temp_很{template}。.csv',index_col=0)
    # df_neg = pd.read_csv(f'./datasets_ppl_score_period/{task}_ppl_temp_不{template}！.csv',index_col=0)
    # df_pos = pd.read_csv(f'./datasets_ppl_score_period/{task}_ppl_temp_很{template}！.csv',index_col=0)
    ppls[template] = {0:df_neg, 1:df_pos}

In [67]:
candi_templates = []
if mode=='instance':
    ids = accs['满意'].index.tolist()
    preds = []
    for id in ids:
        sent_ppl = {}
        for template in templates:
            neg_ppl = ppls[template][0].loc[id]['ppl']
            pos_ppl = ppls[template][1].loc[id]['ppl']
            mean_ppl = (neg_ppl+pos_ppl)/2
            sent_ppl[template] = mean_ppl
        
        candi_template = min(sent_ppl, key=sent_ppl.get)
        if random_flag:
            candi_template = random.choice(templates)
        candi_templates.append(candi_template)

        neg_logit, pos_logit = accs[candi_template].loc[id]['neg_logits'], accs[candi_template].loc[id]['pos_logits']

        if neg_logit>pos_logit:
            preds.append(0)
        else:
            preds.append(1)

else:
    data_ppl = {}
    preds = []
    for template in templates:
        neg_ppl = ppls[template][0]['ppl'].tolist()
        pos_ppl = ppls[template][1]['ppl'].tolist()
        dataset_ppl = []
        for (neg, pos) in zip(neg_ppl, pos_ppl):
            dataset_ppl.append((neg+pos)/2)

        data_ppl[template] = np.mean(dataset_ppl)

    candi_template = min(data_ppl, key=data_ppl.get)
    neg_logit, pos_logit = accs[candi_template]['neg_logits'].tolist(), accs[candi_template]['pos_logits'].tolist()
    
    for (neg, pos) in zip(neg_logit, pos_logit):
        if neg>pos:
            preds.append(0)
        else:
            preds.append(1)




In [68]:
test_labels = accs['满意'].labels.tolist()
acc = accuracy_score(test_labels, preds)
print(acc)

0.5986842105263158


In [69]:
for template in templates:
    print(f'{template},{candi_templates.count(template)}')

满意,282
可以,241
好,7
行,1
喜欢,340
高兴,14
开心,27


In [36]:
len(candi_templates)

2218